In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, GRU, Bidirectional, Dense, RepeatVector, TimeDistributed


import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('TurinAHU.csv')
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Extracting day, month, year, and time into separate columns
df["day"] = df["Timestamp"].dt.day
df["month"] = df["Timestamp"].dt.month
df["year"] = df["Timestamp"].dt.year
df["time"] = df["Timestamp"].dt.hour * 3600 + df["Timestamp"].dt.minute * 60 + df["Timestamp"].dt.second

# Dropping the "timestamp" column
df = df.drop("Timestamp", axis=1)

# Reordering the columns
cols = df.columns.tolist()
cols = ["time", "day", "month", "year"] + cols[:-4]
df = df[cols]
df

In [ ]:
############################################################# STATELESS BiLSTM #############################################################

# Scale the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# Prepare the data for training
X = []
y = []
n_future = 2 # number of timesteps to predict
n_past = 96 # number of timesteps to use as input
for i in range(n_past, len(data_scaled) - n_future + 1):
    X.append(data_scaled[i - n_past:i, :])
    y.append(data_scaled[i:i + n_future, :])

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Bidirectional(LSTM(64, activation='tanh', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
#model.add(Bidirectional(LSTM(64, activation='tanh', dropout=0.5, return_sequences=True)))
model.add(Bidirectional(LSTM(64, activation='tanh', dropout=0.5)))
model.add(RepeatVector(n_future))
#model.add(Bidirectional(LSTM(64, activation='tanh', dropout=0.5, return_sequences=True)))
#model.add(Bidirectional(LSTM(64, activation='tanh', dropout=0.5, return_sequences=True)))
model.add(TimeDistributed(Dense(X_train.shape[2], activation='linear')))
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2)

# Evaluate the model
y_pred = model.predict(X_test)
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2]))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(y_test.shape[0]*y_test.shape[1], y_test.shape[2]))

mae = np.mean(np.abs(y_pred_rescaled - y_test_rescaled), axis=0)
mape = np.mean(np.abs((y_pred_rescaled - y_test_rescaled) / y_test_rescaled), axis=0) * 100
rmse = np.sqrt(np.mean(np.square(y_pred_rescaled - y_test_rescaled), axis=0))

print('MAE:', mae)
print('MAPE:', mape)
print('RMSE:', rmse)